In [1]:
!pip list

Package                                 Version
--------------------------------------- -----------
aiohappyeyeballs                        2.6.1
aiohttp                                 3.12.15
aiosignal                               1.4.0
akracer                                 0.0.14
akshare                                 1.17.56
annotated-types                         0.7.0
anyio                                   4.11.0
asttokens                               3.0.0
attrs                                   25.3.0
aws-requests-auth                       0.4.3
beautifulsoup4                          4.13.5
boto3                                   1.40.38
botocore                                1.40.38
certifi                                 2025.8.3
cffi                                    2.0.0
charset-normalizer                      3.4.3
click                                   8.3.0
comm                                    0.2.3
curl_cffi                               0.13.0
debugpy   

# 用 Python 快速获取基金的股票持仓增减情况 | 更新版
 https://mp.weixin.qq.com/s?__biz=MzU1MTM4OTk0MQ==&mid=2247487084&idx=1&sn=b307075af1702d8c1dbda7d9e219a9b5&chksm=fb9351f6cce4d8e0a647c902b25eca89804f8c3169eecd1216f0c5c7d90e64bee900cc7ed2c1&mpshare=1&scene=23&srcid=0309GDbmEHx70KjI19ionXfv&sharer_sharetime=1646819026253&sharer_shareid=a4c6299b7a875e1e5ddbc56b4c71e4dd#rd

In [1]:
import akshare as ak
import pandas as pd
import numpy as np

print(f'pandas version: {pd.__version__}')
print(f'akshare version: {ak.__version__}')

pandas version: 2.3.2
akshare version: 1.17.56


In [2]:
# 易方达蓝筹精选，005827
# 前海开源公共事业股票，005669
years = ['2022','2023','2024']

data = pd.DataFrame()
for yr in years:
    # df_tmp = ak.fund_em_portfolio_hold(code="005827",year=yr) # akshare version: 1.0.91
    df_tmp = ak.fund_portfolio_hold_em(symbol="005827",date=yr) # akshare version: 1.3.41
    # data = data.append(df_tmp)  # Panda 1.0 version
    data = pd.concat([data, df_tmp], ignore_index=True) # Panda 2.0 use concat

data['季度']=data['季度'].apply(lambda x:x[:6])
data['季度'] = data['季度'].str.replace('年','Q')
data['占净值比例'] = pd.to_numeric(data['占净值比例'])

In [3]:
data

,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,1,600519,贵州茅台,9.81,315.30,542000.70,2022Q1
1,2,600036,招商银行,9.74,11500.01,538200.47,2022Q1
2,3,00700,腾讯控股,9.66,1760.00,534124.70,2022Q1
3,4,002415,海康威视,9.35,12600.00,516600.00,2022Q1
4,5,000568,泸州老窖,9.08,2700.00,501876.00,2022Q1
...,...,...,...,...,...,...,...
482,121,603310,巍华新材,0.00,0.03,0.46,2024Q4
483,122,603091,众鑫股份,0.00,0.01,0.42,2024Q4
484,123,603350,安乃达,0.00,0.01,0.38,2024Q4
485,124,603205,健尔康,0.00,0.01,0.35,2024Q4


In [4]:
# 假设咱们需要对比2021年1季度和2021年2季度的持仓变化情况，则只需要在上面的数据上选择 2021Q1 和 2021Q2 即可。
s1 = '2022Q1'
s2 = '2023Q2'
s1_share = s1+'持股数'
s2_share = s2+'持股数'
s1_value = s1+'持仓市值'
s2_value = s2+'持仓市值'
s1_ratio = s1+'持仓比例'
s2_ratio = s2+'持仓比例'

df1 =data[data['季度']==s1]
df1 = df1[['股票代码', '股票名称','持股数','持仓市值','占净值比例']]
df1 = df1.rename(columns={'持股数':s1_share,'持仓市值':s1_value,'占净值比例':s1_ratio})
df2 =data[data['季度']==s2]
df2 = df2[['股票代码', '股票名称','持股数','持仓市值','占净值比例']]
df2 = df2.rename(columns={'持股数':s2_share,'持仓市值':s2_value,'占净值比例':s2_ratio})

df_merge = pd.merge(df1,df2,on='股票代码',how='outer')

In [5]:
df_merge

,股票代码,股票名称_x,2022Q1持股数,2022Q1持仓市值,2022Q1持仓比例,股票名称_y,2023Q2持股数,2023Q2持仓市值,2023Q2持仓比例
0,000001,平安银行,13000.00,199940.00,3.62,NaN,NaN,NaN,NaN
1,000516,NaN,NaN,NaN,NaN,国际医学,3800.00,31958.00,0.65
2,000568,泸州老窖,2700.00,501876.00,9.08,泸州老窖,2180.00,456862.60,9.29
3,000858,五粮液,3236.01,501775.45,9.08,五粮液,2770.00,453088.90,9.21
4,001282,NaN,NaN,NaN,NaN,三联锻造,0.01,0.35,0.00
...,...,...,...,...,...,...,...,...,...
87,688593,NaN,NaN,NaN,NaN,新相微,0.10,1.40,0.00
88,688620,NaN,NaN,NaN,NaN,安凯微,0.10,1.25,0.00
89,688623,NaN,NaN,NaN,NaN,双元科技,0.02,1.54,0.00
90,688629,NaN,NaN,NaN,NaN,华丰科技,0.07,1.32,0.00


In [6]:
# Q2 和 Q4，即半年度和年度报告，是需要披露全部持仓的
# 合并后，在dataframe 中 NaN 的数据应为 0 
if s1.endswith('Q2') or s1.endswith('Q4'):
    df_merge[s1_share] = df_merge[s1_share].fillna(0)
    df_merge[s1_value] = df_merge[s1_value].fillna(0)
    df_merge[s1_ratio] = df_merge[s1_ratio].fillna(0)

if s2.endswith('Q2') or s2.endswith('Q4'):
    df_merge[s2_share] = df_merge[s2_share].fillna(0)
    df_merge[s2_value] = df_merge[s2_value].fillna(0)
    df_merge[s2_ratio] = df_merge[s2_ratio].fillna(0)


df_merge['持股数变化'] = df_merge[s2_share] - df_merge[s1_share]
df_merge = df_merge.sort_values(s2_value,ascending=False)


df_merge['股票名称'] = df_merge['股票名称_y']
# df_merge['股票名称'] = df_merge['股票名称'].fillna('0')
# df_merge.loc[df_merge['股票名称']=='0','股票名称'] = df_merge.loc[df_merge['股票名称']=='0','股票名称_x']
df_merge.loc[df_merge['股票名称'].isna(),'股票名称'] = df_merge.loc[df_merge['股票名称'].isna(),'股票名称_x']
df_merge = df_merge[['股票代码', '股票名称', s1_share, 
                     s1_value, s1_ratio, 
                     s2_share,s2_value, 
                     s2_ratio, '持股数变化']]

In [7]:
df_merge

,股票代码,股票名称,2022Q1持股数,2022Q1持仓市值,2022Q1持仓比例,2023Q2持股数,2023Q2持仓市值,2023Q2持仓比例,持股数变化
14,00700,腾讯控股,1760.00,534124.70,9.66,1600.00,489165.71,9.94,-160.00
56,600519,贵州茅台,315.30,542000.70,9.81,289.00,488699.00,9.93,-26.30
2,000568,泸州老窖,2700.00,501876.00,9.08,2180.00,456862.60,9.29,-520.00
3,000858,五粮液,3236.01,501775.45,9.08,2770.00,453088.90,9.21,-466.01
11,002304,洋河股份,3630.00,492336.90,8.91,3285.00,431484.75,8.77,-345.00
...,...,...,...,...,...,...,...,...,...
10,001367,海森药业,NaN,NaN,NaN,0.01,0.22,0.00,NaN
7,001324,长青科技,NaN,NaN,NaN,0.01,0.22,0.00,NaN
65,603172,万丰股份,NaN,NaN,NaN,0.01,0.20,0.00,NaN
12,002415,海康威视,12600.00,516600.00,9.35,0.00,0.00,0.00,-12600.00


In [8]:
format_dict = {s1_share: '{0:.2f}', s2_share: '{0:.2f}', 
               s1_value: '{0:.0f}', s2_value: '{0:.0f}',
               s1_ratio: '{0:.2f}', s2_ratio: '{0:.2f}', 
               '持股数变化': '{0:.2f}'}

df_merge.style.hide(axis=0)\
                .hide(subset=['股票代码'], axis="columns")\
                .format(format_dict)\
                .bar(subset=['持股数变化'],color=['#99ff66','#ee7621'],align='mid')

股票名称,2022Q1持股数,2022Q1持仓市值,2022Q1持仓比例,2023Q2持股数,2023Q2持仓市值,2023Q2持仓比例,持股数变化
腾讯控股,1760.00,534125,9.66,1600.00,489166,9.94,-160.00
贵州茅台,315.30,542001,9.81,289.00,488699,9.93,-26.30
泸州老窖,2700.00,501876,9.08,2180.00,456863,9.29,-520.00
五粮液,3236.01,501775,9.08,2770.00,453089,9.21,-466.01
洋河股份,3630.00,492337,8.91,3285.00,431485,8.77,-345.00
招商银行,11500.01,538200,9.74,11500.01,376740,7.66,0.00
伊利股份,11600.00,427924,7.74,11600.00,328512,6.68,0.00
中国海洋石油,nan,nan,nan,30500.00,314948,6.40,nan
香港交易所,1658.00,499405,9.04,1115.00,303468,6.17,-543.00
美团-W,2200.00,277625,5.02,2500.00,281895,5.73,300.00


In [9]:
df_merge_update = df_merge[~(df_merge['持股数变化'].isna())]
df_merge_update.style.hide(axis="index")\
                .hide(subset=['股票代码'], axis="columns")\
                .format(format_dict)\
                .bar(subset=['持股数变化'],color=['#99ff66','#ee7621'],align='mid')

股票名称,2022Q1持股数,2022Q1持仓市值,2022Q1持仓比例,2023Q2持股数,2023Q2持仓市值,2023Q2持仓比例,持股数变化
腾讯控股,1760.00,534125,9.66,1600.00,489166,9.94,-160.00
贵州茅台,315.30,542001,9.81,289.00,488699,9.93,-26.30
泸州老窖,2700.00,501876,9.08,2180.00,456863,9.29,-520.00
五粮液,3236.01,501775,9.08,2770.00,453089,9.21,-466.01
洋河股份,3630.00,492337,8.91,3285.00,431485,8.77,-345.00
招商银行,11500.01,538200,9.74,11500.01,376740,7.66,0.00
伊利股份,11600.00,427924,7.74,11600.00,328512,6.68,0.00
香港交易所,1658.00,499405,9.04,1115.00,303468,6.17,-543.00
美团-W,2200.00,277625,5.02,2500.00,281895,5.73,300.00
海康威视,12600.00,516600,9.35,0.00,0,0.00,-12600.00


# 函数封装

将上面的详细过程封装成一个自定义函数，这样咱们就可以在后续重复使用该函数来获取不同基金的持仓变化信息了。


In [17]:
def compare(code,years,s1,s2):
    """
    code,str,基金代码;
    years,list,年份列表,['yr1','yr2','……'];
    s1,str,靠前的季度, 格式为 'YYYYQ1',例如: '2021Q2';
    s2,str,靠后的季度, 格式为 'YYYYQ1',例如: '2021Q2';
    注意，s1和s2的年份应在 years 里
    """
    
    s1_share = s1+'持股数'
    s2_share = s2+'持股数'
    s1_value = s1+'持仓市值'
    s2_value = s2+'持仓市值'
    s1_ratio = s1+'持仓比例'
    s2_ratio = s2+'持仓比例'

    data = pd.DataFrame()
    for yr in years:
        # df_tmp = ak.fund_em_portfolio_hold(code=code,year=yr) # akshare version: 1.0.91
        df_tmp = ak.fund_portfolio_hold_em(symbol=code,date=yr) # akshare version: 1.3.41
        # data = data.append(df_tmp)  # Panda 1.0 version
        data = pd.concat([data, df_tmp], ignore_index=True) # Panda 2.0 use concat

    data['季度']=data['季度'].apply(lambda x:x[:6])
    data['季度'] = data['季度'].str.replace('年','Q')
    data['占净值比例'] = pd.to_numeric(data['占净值比例'])

    df1 =data[data['季度']==s1]
    df1 = df1[['股票代码', '股票名称','持股数','持仓市值','占净值比例']]
    df1 = df1.rename(columns={'持股数':s1_share,'持仓市值':s1_value,'占净值比例':s1_ratio})
    df2 =data[data['季度']==s2]
    df2 = df2[['股票代码', '股票名称','持股数','持仓市值','占净值比例']]
    df2 = df2.rename(columns={'持股数':s2_share,'持仓市值':s2_value,'占净值比例':s2_ratio})
    
    df_merge = pd.merge(df1,df2,on='股票代码',how='outer')

    # Q2 和 Q4，即半年度和年度报告，是需要披露全部持仓的
    # 合并后，在dataframe 中 NaN 的数据应为 0 

    if s1.endswith('Q2') or s1.endswith('Q4'):
        df_merge[s1_share] = df_merge[s1_share].fillna(0)
        df_merge[s1_value] = df_merge[s1_value].fillna(0)
        df_merge[s1_ratio] = df_merge[s1_ratio].fillna(0)

    if s2.endswith('Q2') or s2.endswith('Q4'):
        df_merge[s2_share] = df_merge[s2_share].fillna(0)
        df_merge[s2_value] = df_merge[s2_value].fillna(0)
        df_merge[s2_ratio] = df_merge[s2_ratio].fillna(0)


    df_merge['持股数变化'] = df_merge[s2_share] - df_merge[s1_share]
    df_merge = df_merge.sort_values(s2_value,ascending=False)


    df_merge['股票名称'] = df_merge['股票名称_y']
    # df_merge['股票名称'] = df_merge['股票名称'].fillna('0')
    # df_merge.loc[df_merge['股票名称']=='0','股票名称'] = df_merge.loc[df_merge['股票名称']=='0','股票名称_x']
    df_merge.loc[df_merge['股票名称'].isna(),'股票名称'] = df_merge.loc[df_merge['股票名称'].isna(),'股票名称_x']
    df_merge = df_merge[['股票代码', '股票名称', s1_share, 
                         s1_value, s1_ratio, 
                         s2_share,s2_value, 
                         s2_ratio, '持股数变化']]
    return df_merge

In [18]:
code = "005827"
years = ['2020','2021']
s1 =  '2020Q3' 
s2 = '2021Q1'
s1_share = s1+'持股数'
s2_share = s2+'持股数'
s1_value = s1+'持仓市值'
s2_value = s2+'持仓市值'
s1_ratio = s1+'持仓比例'
s2_ratio = s2+'持仓比例'

df_merge = compare(code,years,s1,s2)

format_dict = {s1_share: '{0:.2f}', s2_share: '{0:.2f}', 
               s1_value: '{0:.0f}', s2_value: '{0:.0f}',
               s1_ratio: '{0:.2f}', s2_ratio: '{0:.2f}', 
               '持股数变化': '{0:.2f}'}

df_merge.style.hide(axis="index")\
                .hide(subset=['股票代码'], axis="columns")\
                .format(format_dict)\
                .bar(subset=['持股数变化'],color=['#99ff66','#ee7621'],align='mid')

股票名称,2020Q3持股数,2020Q3持仓市值,2020Q3持仓比例,2021Q1持股数,2021Q1持仓市值,2021Q1持仓比例,持股数变化
五粮液,1454.71,321491,9.47,3300.00,884334,10.05,1845.29
香港交易所,1012.26,321819,9.48,2286.00,883734,10.04,1273.74
贵州茅台,193.56,322951,9.52,432.00,867888,9.86,238.44
美团-W,850.00,180603,5.32,3370.43,849458,9.65,2520.43
腾讯控股,718.40,322896,9.51,1610.00,830051,9.43,891.60
泸州老窖,2307.85,331292,9.76,2980.00,670561,7.62,672.15
招商银行,nan,nan,nan,11300.00,577430,6.56,nan
海康威视,3800.01,144818,4.27,9000.02,503101,5.72,5200.01
洋河股份,2585.00,323099,9.52,2500.02,411753,4.68,-84.98
平安银行,nan,nan,nan,13000.00,286130,3.25,nan


# 多支基金
同时获取多支基金的十大股票持仓信息时，在这里只获取每个季度股票名称的信息。

首先，来获取单支基金的十大持仓股票名称信息，自定义函数如下：

In [21]:
# 获取单只基金的十大股票名称信息
def fund_stock_holding(years,code):
    data = pd.DataFrame()
    for yr in years:
        df_tmp = ak.fund_portfolio_hold_em(symbol=code,date=yr) # akshare version:
        # data = data.append(df_tmp)  # Panda 1.0 version
        data = pd.concat([data, df_tmp], ignore_index=True) # Panda 2.0 use concat
        

#     data['季度']=data['季度'].apply(lambda x:x[:8])
    data['季度']=data['季度'].apply(lambda x:x[:6])
    data['季度'] = data['季度'].str.replace('年','Q')
    data['占净值比例'] = pd.to_numeric(data['占净值比例'])    
    data = data.sort_values(['季度','持仓市值'],ascending=[True,False])
    
    # 将序号按持仓市值排序，从大到小    
    data1 = pd.DataFrame()
    quarter = data['季度'].unique().tolist()
    for q in quarter:
        df_tmp = data.query('季度==@q').copy()
        df_tmp['序号'] = list(range(1,len(df_tmp)+1)) 
        data1 = pd.concat([data1, df_tmp], ignore_index=True) # Panda 2.0 use concat
        
    df = data1.set_index(['序号','季度']).stack().unstack([1,2]).head(10)
    # df1.loc[:,(slice(None), ['股票名称','占净值比例'])]
    df = df.loc[:,(slice(None), '股票名称')]  # 只选取 股票名称
    df = df.droplevel(None,axis=1)
    df.columns.name=None
    df = df.reset_index()
#     df.index.name = None
    df['基金代码'] = code
    cols = df.columns.tolist()
    cols = cols[:1] + cols[-1:] + cols[1:-1]  # 将基金代码列名放前面
    df = df[cols]
    return df

In [22]:
years = ['2020','2021']
df3 = fund_stock_holding(years,'005827')
df3

,序号,基金代码,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4
0,1,005827,腾讯控股,香港交易所,泸州老窖,贵州茅台,五粮液,香港交易所,泸州老窖,腾讯控股
1,2,005827,贵州茅台,腾讯控股,洋河股份,美团-W,香港交易所,五粮液,贵州茅台,海康威视
2,3,005827,泸州老窖,五粮液,贵州茅台,洋河股份,贵州茅台,腾讯控股,五粮液,泸州老窖
3,4,005827,五粮液,贵州茅台,腾讯控股,腾讯控股,美团-W,贵州茅台,海康威视,贵州茅台
4,5,005827,中国生物制药,泸州老窖,香港交易所,五粮液,腾讯控股,泸州老窖,香港交易所,五粮液
5,6,005827,香港交易所,美团-W,五粮液,香港交易所,泸州老窖,洋河股份,洋河股份,洋河股份
6,7,005827,华兰生物,中国生物制药,美团-W,泸州老窖,招商银行,美团-W,腾讯控股,香港交易所
7,8,005827,药明生物,药明生物,爱尔眼科,海康威视,海康威视,招商银行,招商银行,招商银行
8,9,005827,石药集团,天坛生物,中国生物制药,爱尔眼科,洋河股份,海康威视,伊利股份,伊利股份
9,10,005827,上海机场,爱尔眼科,海康威视,颐海国际,平安银行,招商银行,平安银行,平安银行


In [24]:
# 通过自定义函数 fund_stock_holding(years,code) 可以同时获取多支基金的十大持仓信息，代码如下：

codes = ['005669','006281','003956','001532'] #,
        #  '001714','001054','001809','003853',
        #  '004890','001125','001808']
df = pd.DataFrame()
for code in codes:
    df_tmp = fund_stock_holding(years,code)
    df = pd.concat([df, df_tmp], ignore_index=True) # Panda 2.0 use concat
df

,序号,基金代码,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4
0,1,005669,长江电力,长江电力,晶澳科技,比亚迪,中科电气,比亚迪股份,华润电力,华润电力
1,2,005669,华测检测,华测检测,信义光能,中科电气,永兴材料,亿纬锂能,亿纬锂能,华能国际电力股份
2,3,005669,中国移动,国投电力,隆基绿能,亿纬锂能,东方日升,中科电气,比亚迪股份,中国电力
3,4,005669,隆基绿能,上海机场,长江电力,星源材质,星源材质,宁德时代,华能国际电力股份,亿纬锂能
4,5,005669,川投能源,川投能源,中科电气,阳光电源,亿纬锂能,天合光能,法拉电子,比亚迪股份
5,6,005669,深圳高速公路股份,深圳燃气,宏川智慧,永兴材料,诺德股份,法拉电子,中国电力,中广核新能源
6,7,005669,桂冠电力,内蒙华电,上海机场,诺德股份,鹏辉能源,东方日升,宁德时代,法拉电子
7,8,005669,粤海投资,华电国际,通威股份,固德威,天合光能,星源材质,中广核新能源,龙源电力
8,9,005669,国投电力,瀚蓝环境,阳光电源,鹏辉能源,晶澳科技,鹏辉能源,中科电气,比亚迪
9,10,005669,上海机场,深圳高速公路股份,瀚蓝环境,天合光能,宁德时代,融捷股份,新天绿色能源,新天绿色能源


In [26]:
# 获取基金名称信息
df_fund_info = ak.fund_name_em()
df_fund_info = df_fund_info.query('基金代码 in @codes')
df_fund_info = df_fund_info[['基金代码','基金简称']]
df_fund_info = df_fund_info.reset_index(drop=True)
df_fund_info

,基金代码,基金简称
0,001532,华安文体健康混合A
1,003956,南方产业智选股票A
2,005669,前海开源公用事业股票
3,006281,万家人工智能混合A


In [27]:
# 有了基金名称信息后，可以用 pandas 通过拼接的方式，将基金名称信息显示出来，代码如下：
df_fund = pd.merge(df,df_fund_info,on='基金代码')
cols = df_fund.columns.tolist()
cols = cols[:1] + cols[-1:] + cols[2:-1]  # 将基金简称放前面
df_fund =df_fund[cols]
df_fund

,序号,基金简称,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4
0,1,前海开源公用事业股票,长江电力,长江电力,晶澳科技,比亚迪,中科电气,比亚迪股份,华润电力,华润电力
1,2,前海开源公用事业股票,华测检测,华测检测,信义光能,中科电气,永兴材料,亿纬锂能,亿纬锂能,华能国际电力股份
2,3,前海开源公用事业股票,中国移动,国投电力,隆基绿能,亿纬锂能,东方日升,中科电气,比亚迪股份,中国电力
3,4,前海开源公用事业股票,隆基绿能,上海机场,长江电力,星源材质,星源材质,宁德时代,华能国际电力股份,亿纬锂能
4,5,前海开源公用事业股票,川投能源,川投能源,中科电气,阳光电源,亿纬锂能,天合光能,法拉电子,比亚迪股份
5,6,前海开源公用事业股票,深圳高速公路股份,深圳燃气,宏川智慧,永兴材料,诺德股份,法拉电子,中国电力,中广核新能源
6,7,前海开源公用事业股票,桂冠电力,内蒙华电,上海机场,诺德股份,鹏辉能源,东方日升,宁德时代,法拉电子
7,8,前海开源公用事业股票,粤海投资,华电国际,通威股份,固德威,天合光能,星源材质,中广核新能源,龙源电力
8,9,前海开源公用事业股票,国投电力,瀚蓝环境,阳光电源,鹏辉能源,晶澳科技,鹏辉能源,中科电气,比亚迪
9,10,前海开源公用事业股票,上海机场,深圳高速公路股份,瀚蓝环境,天合光能,宁德时代,融捷股份,新天绿色能源,新天绿色能源
